<a href="https://colab.research.google.com/github/andreacangiani/NSPDE-ANA2024/blob/main/Python/CP8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FEniCS Tutorial 2: solving diffusion problems in 2D

We start by loading the FEniCSx modules, exaclty as in the previous tutorial.

In [ ]:
try:
    import dolfinx
except ImportError:
    !wget "https://github.com/fem-on-colab/fem-on-colab.github.io/raw/a3d664e/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx

In [ ]:
try:
    import viskex
except ImportError:
    !pip3 install "viskex@git+https://github.com/viskex/viskex.git@f26b326"
    import viskex

In [ ]:
import dolfinx.fem
import dolfinx.fem.petsc
import dolfinx.mesh
import mpi4py.MPI
import numpy as np
import ufl
import viskex

We consider the model boundary value problem: find $u: \Omega = (0, 1)^2 \to \mathbb{R}$ such that
\begin{equation*}
\begin{cases}
- \Delta u = f, & \text{in } \Omega,\\
u = g, & \text{on } \partial\Omega.
\end{cases}
\end{equation*}

with $f\equiv 1$ and the boundary value $g(\mathbf{x})$ given by
$$
g(\mathbf{x}) = g(x_0, x_1) = \sin(3 \pi x_0 + 1) \ \sin(3 \pi x_1 + 1).
$$


**Task 1: create a mesh.** As first example, we generate a triangular mesh of the domain $\Omega$, dividing both the horizontal and vertical sides of the square in nxm equispaced subintervals.

Similarly to 1D case, `dolfinx.mesh` provides the function `create_unit_square` for this task.

In [ ]:
n = 10
m = 15
mesh = dolfinx.mesh.create_unit_square(mpi4py.MPI.COMM_WORLD, n, m)

Let's see how many cells are in the mesh.

We store in:
* tdim: the problem dimension (2)
* fdim: the mesh scheleton entities dimension (1)
* num_cells: the number of triangles in the mesh

We can extract the num_cells information as we did in the 1D case from the connectivity or directly...

In [ ]:
tdim =
fdim =

connectivity_cells_to_vertices = mesh.topology.connectivity(mesh.topology.dim,0)
connectivity_cells_to_vertices
num_cells = len(connectivity_cells_to_vertices)
num_cells

num_cells = mesh.topology.index_map(tdim).size_local
num_cells

We can obtain an interactive plot of the domain using `pyvista`. (Click on the menu: next to the dropdown that contains "Geometry 0" you may find three different representations: the domain itself, the edges of the mesh, and both overlayed.)

In [ ]:
viskex.dolfinx.plot_mesh(mesh)

**Task 2:** Determine IDs of boundary edges in view of the application of the Dirichlet boundary condition.

As in 1D case, this is obtained via the `dolfinx.mesh` `locate_entities_boundary` function. We want all edges on the boundary, but the function always requires a third input to permit the selection of parts of the boundary. As a workaround to this, we pass as third argument the function which always returns `True`.

In [ ]:
boundary_entities = dolfinx.mesh.locate_entities_boundary(
    mesh, fdim, lambda x: np.full((x.shape[1], ), True))
boundary_entities

We can store the boundary entites in a `dolfinx.mesh` `MeshTags` object, and plot it using `viskex` to check this was done correctly.

In [ ]:
boundaries = dolfinx.mesh.meshtags(
    mesh, mesh.topology.dim - 1, boundary_entities, np.ones_like(boundary_entities))

viskex.dolfinx.plot_mesh_tags(mesh, boundaries, "boundaries")

**Task 3: create FEM space.**

Define the finite element function space $V_h$ using $\mathbb{P}_1$ Lagrange elements.

This part of the code is indistinguishable from the 1D version...

In [ ]:
Vh =

... and compute its dimension

In [ ]:
Vh_dim =
Vh_dim

Once the FE space is at hand, we introduce ufl symbols to define the trial and test functions for our weak formulation:

In [ ]:
uh =
vh =

**Task 4:** set up FEM system

We produce the weak formulation of the problem: find $u_h\in V_h$ such that
$$ \int_\Omega \nabla u \cdot \nabla v \ \mathrm{d} \mathbf{x} = \int_\Omega f \ v \ \mathrm{d} \mathbf{x},\qquad\forall v_h\in V_h$$
using `ufl`.

In [ ]:
dx =
inner =
grad =
A =

In [ ]:
F =

**Task 5:** set up the boundary conditions

In order to assign the boundary condition we first need to evaluate the expression of $g$ (i.e. $\sin(3 \pi x_0 + 1) \ \sin(3 \pi x_1 + 1)$ in our case) on the finite element space $V_h$. We do this by interpolation, so we define the discrete boundary condition as
$$g_h=I_h g$$

with $I_h$ the interpolation operator.

In [ ]:
gh = dolfinx.fem.Function(Vh)
gh.interpolate(lambda x: np.sin(3 * np.pi * x[0] + 1) * np.sin(3 * np.pi * x[1] + 1))

We then initialize a `dolfinx.fem` `dirichletbc` object, stating that the Dirichlet boundary condition should be equal to `gh` on each facet in `boundary_entities`.

In [ ]:
boundary_dofs = dolfinx.fem.locate_dofs_topological(Vh, fdim, boundary_entities)
bc = dolfinx.fem.dirichletbc(gh, boundary_dofs)

**Task 6:** solve the FEM system.

As in 1D case, we have to first provide a `Function` class to store the solution of a finite element problem and then we are ready to solve the discrete problem allocating a new `LinearProblem` (which uses `PETSc`), providing as input the bilinear form `a`, the linear functional `F`, the boundary conditions `bcs`, and where to store the solution. Further solver options can also be passed to `PETSc`.

In [ ]:
solution =

In [ ]:
problem = dolfinx.fem.petsc.LinearProblem(
    A, F, bcs=[bc], u=solution,
    petsc_options={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})
_ = problem.solve()

**Task7:** plot the solution

In [ ]:
viskex.dolfinx.plot_scalar_field(solution, "u_h")

And here is a3D plot:

In [ ]:
viskex.dolfinx.plot_scalar_field(solution, "u_h", warp_factor=0.1)

# Computing errors

To test convergence, let us consider a problem where we fix the exact solution. Find $u: \Omega \to \mathbb{R}$ such that
\begin{equation*}
\begin{cases}
- \Delta u = f, & \text{in } \Omega,\\
u = g, & \text{on } \partial\Omega,
\end{cases}
\end{equation*}
where
$$\Omega = (0, 1)^2$$
and $f$ and $g$ are chosen such that the exact solution to the strong formulation is
$$u_{\text{ex}}(x, y) = \sin(2 \pi x) \ \cos(2 \pi y).$$

Thus,
$$f := - \Delta u_{\text{ex}} = - \partial_{xx} u_{\text{ex}} - \partial_{yy} u_{\text{ex}} = 8 \pi^2 \ \sin(2 \pi x) \ \cos(2 \pi y).$$
> Furthermore, by splitting the boundary $\partial\Omega$ in the four sides of the square $\Omega$, we have
\begin{equation*}
\begin{cases}
g_1(y) = g(0, y) := u_{\text{ex}}(0, y) = 0, & \text{on } \Gamma_1 = \left\{0\right\} \times [0, 1],\\
g_2(x) = g(x, 0) := u_{\text{ex}}(x, 0) = \sin(2 \pi x), & \text{on } \Gamma_2 = [0, 1] \times \left\{0\right\},\\
g_3(y) = g(1, y) := u_{\text{ex}}(1, y) = 0, & \text{on } \Gamma_3 = \left\{1\right\} \times [0, 1],\\
g_4(y) = g(x, 1) := u_{\text{ex}}(x, 1) = \sin(2 \pi x), & \text{on } \Gamma_4 = [0, 1] \times \left\{1\right\},\\
\end{cases}
\end{equation*}

We want to generate a structured triangular mesh of the domain Ω, dividing both the horizontal and vertical sides of the square in n equispaced subintervals. The function should also locate boundaries Γ1,Γ2,Γ3 and Γ4.

We write the function unit_square_structured_mesh(n) using the builtin dolfinx.mesh.create_unit_square structured mesh generator and the boundary marking through four calls to dolfinx.mesh.locate_entities_boundary.

First we require a few more imports.

In [ ]:
import petsc4py.PETSc
import typing

In [ ]:
def unit_square_structured_mesh(n: int) -> typing.Tuple[dolfinx.mesh.Mesh, typing.Dict[int, np.ndarray]]:
    """Generate a structured mesh of the unit square, and locate its four boundaries."""
    # Generate structured mesh
    mesh = dolfinx.mesh.create_unit_square(mpi4py.MPI.COMM_WORLD, n, n)

    # Locate boundary entities
    boundaries = {
        1: dolfinx.mesh.locate_entities_boundary(mesh, mesh.topology.dim - 1, lambda x: np.isclose(x[0], 0.0)),
        2: dolfinx.mesh.locate_entities_boundary(mesh, mesh.topology.dim - 1, lambda x: np.isclose(x[1], 0.0)),
        3: dolfinx.mesh.locate_entities_boundary(mesh, mesh.topology.dim - 1, lambda x: np.isclose(x[0], 1.0)),
        4: dolfinx.mesh.locate_entities_boundary(mesh, mesh.topology.dim - 1, lambda x: np.isclose(x[1], 1.0))
    }

    # Return
    return mesh, boundaries

Next we write the function `unit_square_poisson_solve(mesh, boundaries, order)` combining the usual steps that we carry out when solving a finite element problem with FEniCS: (i) definition of a finite element space, (ii) definition of the weak form, (iii) definition of the boundary conditions, (iv) linear system assembly, and (v) linear system solve.

For the solving fase, we access the library KSP of linear system solvers from PETSc

In [ ]:
def unit_square_poisson_solve(
    mesh: dolfinx.mesh.Mesh, boundaries: typing.Dict[int, np.ndarray], order: int
) -> typing.Tuple[dolfinx.fem.functionspace, petsc4py.PETSc.Mat, dolfinx.fem.Function]:
    """Solve a Poisson problem on the unit square."""
    # Function space
    Vh = dolfinx.fem.functionspace(mesh, ("Lagrange", order))

    # Weak form
    uh = ufl.TrialFunction(Vh)
    vh = ufl.TestFunction(Vh)
    x = ufl.SpatialCoordinate(mesh)
    f = 8 * ufl.pi**2 * ufl.sin(2 * ufl.pi * x[0]) * ufl.cos(2 * ufl.pi * x[1])
    dx = ufl.dx
    inner = ufl.inner
    grad = ufl.grad
    a = inner(grad(uh), grad(vh)) * dx
    F = f * vh * dx

    # Boundary conditions
    boundary_value = dolfinx.fem.Function(Vh)
    boundary_value.interpolate(lambda x: np.sin(2 * np.pi * x[0]))
    zero = dolfinx.fem.Constant(mesh, 0.0)
    boundary_dofs = {
        i: dolfinx.fem.locate_dofs_topological(Vh, mesh.topology.dim - 1, boundaries[i]) for i in range(1, 5)}

    bcs = [
        dolfinx.fem.dirichletbc(zero, boundary_dofs[1], Vh),
        dolfinx.fem.dirichletbc(boundary_value, boundary_dofs[2]),
        dolfinx.fem.dirichletbc(zero, boundary_dofs[3], Vh),
        dolfinx.fem.dirichletbc(boundary_value, boundary_dofs[4])]

    # Assemble system
    a_cpp = dolfinx.fem.form(a)
    F_cpp = dolfinx.fem.form(F)
    A = dolfinx.fem.petsc.assemble_matrix(a_cpp, bcs)
    A.assemble()
    b = dolfinx.fem.petsc.assemble_vector(F_cpp)
    dolfinx.fem.petsc.apply_lifting(b, [a_cpp], [bcs])
    dolfinx.fem.petsc.set_bc(b, bcs)

    # Solve
    solution = dolfinx.fem.Function(Vh)
    ksp = petsc4py.PETSc.KSP()
    ksp.create(mesh.comm)
    ksp.setOperators(A)
    ksp.setType("preonly")
    ksp.getPC().setType("lu")
    ksp.getPC().setFactorSolverType("mumps")
    ksp.setFromOptions()
    ksp.solve(b, solution.vector)

    # Return
    return Vh, A, solution

Let's test it

In [ ]:
Vh, A, solution = unit_square_poisson_solve(mesh, boundaries, 1)
viskex.dolfinx.plot_scalar_field(solution, "u_h")

Finally, the main task: computing errors.

To this end, we write a function `unit_square_solution_error(mesh, solution, space)` that computes the norm of the error between a finite element solution and the exact solution to the Poisson problem on the domain $\Omega$. The first input argument `mesh` contains the mesh of $\Omega$, the second input argument `solution` contains the finite element solution, while the norm definition is derived from the third input argument `space` (if `space == 0`, the $L^2(\Omega) = H^0(\Omega)$ norm is used, while if `space == 1` the $H^1(\Omega)$ norm is used).

The function implments the following steps: (i) definition of the exact solution, (ii) computation of the difference between the finite element solution and the exact solution, (iii) representation of the square of norm of the error in `ufl`, (iv) evaluation of the square of norm of the error by assembling the `ufl` representation, and (v) application of a square root (do not forget this!) to get the error from its square.

> Indented block



In [ ]:
def unit_square_solution_error(
    mesh: dolfinx.mesh.Mesh, solution: dolfinx.fem.Function, space: int
) -> float:
    """Compute the error between the FE solution and the exact solution."""
    # Definition of the exact solution
    x = ufl.SpatialCoordinate(mesh)
    u_ex = ufl.sin(2 * ufl.pi * x[0]) * ufl.cos(2 * ufl.pi * x[1])

    # Computation of the difference between the finite element solution and the exact solution
    diff = solution - u_ex

    # UFL representation of the square of the norm of the error depending on the input argument space
    dx = ufl.dx
    if space == 0:
        eh_squared_ufl = diff * diff * dx
    elif space == 1:
        inner = ufl.inner
        grad = ufl.grad
        eh_squared_ufl = diff * diff * dx + inner(grad(diff), grad(diff)) * dx
    else:
        raise RuntimeError("Invalid space.")

    # Evaluation of the square of the norm of the error by assembling the UFL representation
    eh_squared = dolfinx.fem.assemble_scalar(dolfinx.fem.form(eh_squared_ufl))

    # Compute the square root and return
    return np.sqrt(eh_squared)

We test the implementation on the solution we have obtained from the mesh with $n = 10$ and $\mathbb{P}^1$ finite elements.

Here is the L2 error

In [ ]:
unit_square_solution_error(mesh, solution, 0)

And the H1 error

In [ ]:
unit_square_solution_error(mesh, solution, 1)

**Exercise 1:** Check the rate of convergence by running a series of experiments. (See, eg.  CP5)

# Working with subdomains

We now solve the same problem but with the forcing function given by:

$$
f(\mathbf{x}) = \begin{cases}
50, & \mathbf{x} \in [0.2, 0.8]^2,\\
1, & \text{otherwise},
\end{cases}
$$


We shall code this using by defining two subdomains:
$$\Omega_2 = [0.2, 0.8]^2 \quad \text{and} \quad \Omega_1 = \Omega \setminus \Omega_0$$

 In order to do so, we need to ensure that the mesh is alligned with the subdomains. For instance, we can take $n=m=10$:

In [ ]:
n = 10
m = 10
mesh = dolfinx.mesh.create_unit_square(mpi4py.MPI.COMM_WORLD, n, m)

num_cells = mesh.topology.index_map(tdim).size_local
num_cells

In [ ]:
viskex.dolfinx.plot_mesh(mesh)

In [ ]:
boundary_entities = dolfinx.mesh.locate_entities_boundary(
    mesh, fdim, lambda x: np.full((x.shape[1], ), True))
boundary_entities

**New Task:** Define the two subdomains $\Omega_1$ and $\Omega_2$ in view of the implementation of the forcing function $f$.

*   mark $\Omega_1$ and $\Omega_2$ with labels `1` and `2`
*   use the `dolfinx.mesh` function `locate_entities` to determine whether a cell is in $\Omega_2$. This function checks each of the three vertices of the triangular cell, and locates all cells in which the provided condition is satisfied on all three vertices.

In [ ]:
inner_subdomain_entities = dolfinx.mesh.locate_entities(
    mesh, mesh.topology.dim, lambda x: np.logical_and(
        np.logical_and(x[0] >= 0.2, x[0] <= 0.8),
        np.logical_and(x[1] >= 0.2, x[1] <= 0.8)))
inner_subdomain_entities

We label each cell in $\Omega_2$ with the label `2` by using the `np.full` function which return a new array with shape of input filled with value.

In [ ]:
inner_subdomains_labels = np.full(inner_subdomain_entities.shape, 2, dtype=np.intc)
inner_subdomains_labels

The remaining cells will belong to $\Omega_1$. Recall that in num_cells we have stored the total number of cells


In [ ]:
outer_subdomain_entities = np.setdiff1d(np.arange(num_cells), inner_subdomain_entities)
outer_subdomain_entities

In [ ]:
outer_subdomains_labels = np.full(outer_subdomain_entities.shape, 1, dtype=np.intc)
outer_subdomains_labels

We then store both subdomains in a `dolfinx.mesh` `MeshTags` object.

In [ ]:
subdomains_entities_unsorted = np.hstack((outer_subdomain_entities, inner_subdomain_entities)).astype(np.int32)
subdomains_values_unsorted = np.hstack((outer_subdomains_labels, inner_subdomains_labels)).astype(np.int32)
subdomains_entities_argsort = np.argsort(subdomains_entities_unsorted)
subdomains_entities_sorted = subdomains_entities_unsorted[subdomains_entities_argsort]
subdomains_values_sorted = subdomains_values_unsorted[subdomains_entities_argsort]
subdomains = dolfinx.mesh.meshtags(
    mesh, mesh.topology.dim, subdomains_entities_sorted, subdomains_values_sorted)

We finally plot with `pyvista` the subdomains to verify the correct assignment of the label.

In [ ]:
viskex.dolfinx.plot_mesh_tags(mesh, subdomains, "subdomains")

Defining the FEM space is as before:

In [ ]:
Vh = dolfinx.fem.functionspace(mesh, ("Lagrange", 1))

Vh_dim = Vh.dofmap.index_map.size_local
Vh_dim

In [ ]:
uh = ufl.TrialFunction(Vh)
vh = ufl.TestFunction(Vh)

In order to write this problem in `ufl`, we need to inform the integral measure `dx` of the subdomain labels, using `ufl.Measure` and providing as data the `subdomains` object that we have created.

In [ ]:
dx = ufl.Measure("dx")(subdomain_data=subdomains)

The bilinear form is straighforward:

In [ ]:
inner = ufl.inner
grad = ufl.grad
A = inner(grad(uh), grad(vh)) * dx

> We further define the linear functional
> $$ F(v) = \int_\Omega f \ v \ \mathrm{d}\mathbf{x} = \int_{\Omega_1} v \ \mathrm{d}\mathbf{x} + \int_{\Omega_2} 10 \ v \ \mathrm{d}\mathbf{x}.$$
> In order to implement this in `ufl`, we use `dx(1)` to define integration over $\Omega_1$, and similarly `dx(2)` to define integration over $\Omega_2$.

In [ ]:
F = vh * dx(1) + 50 * vh * dx(0)

The rest is as before:

In [ ]:
gh = dolfinx.fem.Function(Vh)
gh.interpolate(lambda x: np.sin(3 * np.pi * x[0] + 1) * np.sin(3 * np.pi * x[1] + 1))

boundary_dofs = dolfinx.fem.locate_dofs_topological(Vh, fdim, boundary_entities)
bc = dolfinx.fem.dirichletbc(gh, boundary_dofs)

solution = dolfinx.fem.Function(Vh)

problem = dolfinx.fem.petsc.LinearProblem(
    A, F, bcs=[bc], u=solution,
    petsc_options={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})
_ = problem.solve()

In [ ]:
viskex.dolfinx.plot_scalar_field(solution, "u_h")

**Exercise 2:** Use the subdomain idea to solve the problem: find $u: \Omega = (0, 1)^2 \to \mathbb{R}$ such that
\begin{equation*}
\begin{cases}
- \nabla (\kappa \nabla u) = f, & \text{in } \Omega,\\
u = g, & \text{on } \partial\Omega.
\end{cases}
\end{equation*}
with $f$  and $g$ as before and
$$
\kappa(\mathbf{x}) = \begin{cases}
1, & \mathbf{x} \in [0.2, 0.8]^2,\\
0.1, & \text{otherwise},
\end{cases}
$$


**Exercise 3:** By yourself or using the official tutorial

https://jorgensd.github.io/dolfinx-tutorial/index.html

as reference, find out how to use the subdomain idea to define different boundary conditions. For instance, solve with:
$$
u = g \quad\text{if } \quad x=0 \quad \text{or}\quad  x=1,
$$
and
$$
-\frac{\partial u}{\partial {\mathbf n}} = h \quad \text{otherwise}.
$$
with
$$
h(x,y)=
\left\{
\begin{array}{ll}
0 & \text{if } y=0\\
-4 & \text{if } y=1.
\end{array}
\right.
$$

